# 0) 데이터 전처리는 무엇인가 (큰 그림부터 잡기)


.median()

최빈값.mode()
.describe()

## 0.1) 데이터 분석 전체 흐름의 정확한 이해
데이터 분석은 보통 다음 순서로 진행된다.
1. 데이터 수집(파일/DB/API)
2. 전처리(정제, 형변환, 결측치, 이상치, 파생변수)
3. 탐색(EDA: 분포, 관계, 패턴, 가설)
4. 시각화(의사소통 + 패턴 확인)
5. 모델링(예측/분류/추천 등)
6. 결과 해석/의사결정

이 순서는 권장 사항이 아니라 필수 순서다.
- 전처리가 끝나지 않으면 그 이후 단계는 정상적으로 작동하지 않기 때문이다

## 0.2) 전처리는 "준비 단계"가 아닌 "전제 조건"이다
전처리는 데이터가 계산·비교·집계·시각화가 가능하도록 구조와 타입을 맞추는 작업이다.

### 전처리를 안 하면 발생하는 문제
1. 계산이 불가능해진다
    - 숫자로 계산해야 할 값이 문자열이면 곱셈, 합계, 평균이 아예 동작하지 않는다
2. 결과의 외곡
    - 같은 의미의 값이 다른 값으로 인식되면, 집계 결과가 쪼개진다
3. 시각화/모델 오류
    - 그래프가 안 그려지거나 모델 학습 단계에서 오류가 난다

기억하자. 데이터 분석 시간의 절반 이상이 전처리다

## 0.3 전처리는 기술이 아니라 사고 과정
전처리는 단순히 astype, fillna, dropna 같은 함수를 외워서 쓰는 작업이 아니다. <br>
전처리의 출발점은 항상 "이 데이터로 어떤 질문에 답하려는가?" 이다.

같은 컬럼이라도
- 매출을 합계로 볼지
- 평균으로 볼지
- 날짜별 추이를 볼지에 따라

필요한 dtype, 결측치 처리 방식, 파생 피처가 전부 달라진다.

## 0.4 전처리 사고 순서 요약
전처리는 감으로 하지 않고, 항상 같은 질문 순서로 진행한다.

### 1. 구조 확인
이 데이터가 다음과 같은지 확인한다.
- 몇 행, 몇 열인지
- 어떤 컬럼들로 구성되어 있는지
- 테이블 형태가 맞는지

<br>

### 2. dtype 확인<br>
각 컬럼이 숫자, 문자열, 날짜인지 확인한다. <br>
이 단계에서 지금 이상태로 계산이 가능한가? 를 확인한다.


<br>

### 3. 값 정리
dtype가 맞지 않거나, 값의 표기가 섞여 있거나, 결측치가 있으면 이 단계에서 정리한다.
- 문자열 정리
- 숫자 파싱
- 결측치 처리
- 범주값 통일

전처리의 대부분은 이 단계에서 발생한다.

<br>

### 4. 분석 가능한 상태로 만들기
마지막으로 다음 사항을 점검한다.
- 필요한 컬럼만 남기고
- 파생 피처를 만들고
- 바로 groupby / 시각화가 가능한지

이 상태가 되서야 "이제 분석을 시작해도 되는 데이터"가 된다.

# 1. Pandas 데이터 구조 이해 (전처리의 재료)
Pandas는 복잡해 보이지만 핵심 데이터 구조는 두 개뿐이다.
- Series
- DataFrame

Pandas에서 하는 모든 연산은 결국 Series 또는 DataFrame을 대상으로 한다.

## 1.1) DataFrame부터 정확히 정의하자
DataFrame은 행(row)과 열(column)로 이루어진 2차원 표 형태의 데이터 구조다.<br>
표라는 말은 결과 모습일 뿐 내부 구조는 Series들의 묶음이다

현실 세계의 데이터는 대부분
- 한 행이 하나의 관측 대상이고
- 한 열이 하나의 속성(변수)로 구성된다.

Pandas의 DataFrame은 이 구조를 그대로 표현하기 때문에 데이터 분석과 리포트의 기본 단위로 사용된다.


## 1.2) Series도 정의하자
Series는 하나의 컬럼을 표현하는 1차원 데이터 구조다. <br>
DataFrame 안의 각 컬럼은 모두 하나의 Series로 구성되어 있다.


즉, DataFrame은 여러 개의 Series가 같은 인덱스를 공유하며 묶여 있는 구조라고 볼 수 있다.


## 1.3) DataFrame vs Series 차이가 중요한 이유
DataFrame과 Series는 겉보기에는 비슷하지만 동작 방식과 결과 형태가 다르다.

이 차이를 이해하지 못하면
- 메서드 결과가 예상과 다르게 나오거나
- 다음 단계 코드에서 오류가 발생하거나
- 집계·결합 단계에서 흐름이 끊긴다.

때문에 전처리를 시작하기 전에는 지금 내가 다루고 있는 게 DataFrame인가, Series인가를 항상 확인해야 한다.

### 코드로 확인해보자

In [177]:
import pandas as pd

df = pd.DataFrame({
    "date": ["2026-01-01", "2026-01-01", "2026-01-02", "2026-01-03"],
    "menu": ["Americano", "Latte", "Latte", "Mocha"],
    "price": [4500, 5000, None, 5500],
    "qty": [2, 1, 2, None],
    "paid": ["TRUE", "True", "FALSE", True],
})

df

,date,menu,price,qty,paid
0,2026-01-01,Americano,4500.0,2.0,TRUE
1,2026-01-01,Latte,5000.0,1.0,True
2,2026-01-02,Latte,NaN,2.0,FALSE
3,2026-01-03,Mocha,5500.0,NaN,True


In [178]:
print("테이블 출력")
print(df)

print("\n테이블 타입 확인")
print(type(df))

테이블 출력
         date       menu   price  qty   paid
0  2026-01-01  Americano  4500.0  2.0   TRUE
1  2026-01-01      Latte  5000.0  1.0   True
2  2026-01-02      Latte     NaN  2.0  FALSE
3  2026-01-03      Mocha  5500.0  NaN   True

테이블 타입 확인
<class 'pandas.core.frame.DataFrame'>


In [179]:
s = df["paid"]

print("시리즈 출력")
print(s)

print("\n시리즈의 타입 확인")
print(type(s))

시리즈 출력
0     TRUE
1     True
2    FALSE
3     True
Name: paid, dtype: object

시리즈의 타입 확인
<class 'pandas.core.series.Series'>


## 1.4) 대괄호 한 개 vs 두 개의 차이
컬럼을 선택할 때 대괄호를 한 개 쓰면 Series가 되고, 대괄호를 두 개 쓰면 DataFrame이 된다. <br>
이 차이는 단순한 형태 차이가 아닌 이후 연산 가능성과 코드 흐름에 직접적인 영향을 준다.
- Series는 단일 컬럼 계산에 유리하고
- DataFrame은 표 형태 유지와 후속 작업에 유리하다.

In [180]:
# 컬럼 하나 → Series
print(type(df["paid"]))

# 컬럼 하나짜리 표 → DataFrame
print(type(df[["paid"]]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


## 1.5) 전처리가 Series 단위로 이루어지는 이유
전처리에서 다루는 대부분의 문제는 컬럼 하나의 문제다.
- dtype이 잘못된 컬럼
- 결측치가 있는 컬럼
- 문자열 표기가 흔들리는 컬럼

이런 문제들은 항상 “한 컬럼(Series)” 단위에서 발생한다.<br>
그래서 전처리는 DataFrame 전체를 한 번에 고치는 작업이 아니라, 문제가 있는 Series를 하나씩 정리하는 과정으로 진행된다.


# 2) 데이터 구조 빠른 점검 루틴

## 2.1) 반드시 확인해야 하는 것
이 세 가지는 EDA에서 가장 먼저 확인해야 하는 최소 점검 세트다. <br>
이 단계의 목적은 데이터를 분석하기 전에 “이 데이터를 다룰 수 있는 상태인가?”를 빠르게 판단하는 것이다.

- 데이터 규모는 적절한가
- 컬럼 구조는 정상적인가 (결측치 유무)
- dtype이 분석 의도와 맞는가

이 세 가지만 확인해도 전처리 방향의 절반은 결정된다 할수 있다.


### 코드 설명
1. info(): 데이터의 전체 구조를 한 번에 훑어보는 용도다. 전체 행 수, 컬럼 수, dtype, 결측치를 한 번에 파악한다.
2. shape: 데이터의 크기를 (행, 열) 형태로 빠르게 확인한다. (열 개수가 1인 경우 Series일 '가능성'이 있음)
3. dtypes: 각 컬럼의 자료형을 개별적으로 확인한다. (info()에 포함된 정보)

In [181]:
print("데이터 프레임의 종합 정보 확인 (행 수, 컬럼 수, dtype, 결측치)")
df.info()

print("\n데이터 프레임의 규모 확인 (행, 열)")
print(df.shape)

print("\n컬럼별 dtype 확인")
print(df.dtypes)

데이터 프레임의 종합 정보 확인 (행 수, 컬럼 수, dtype, 결측치)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    4 non-null      object 
 1   menu    4 non-null      object 
 2   price   3 non-null      float64
 3   qty     3 non-null      float64
 4   paid    4 non-null      object 
dtypes: float64(2), object(3)
memory usage: 288.0+ bytes

데이터 프레임의 규모 확인 (행, 열)
(4, 5)

컬럼별 dtype 확인
date      object
menu      object
price    float64
qty      float64
paid      object
dtype: object


## 2.2) 필요에 따라 해야 하는 것
```python
df.index
df.columns # (Series일경우 df.column 사용한다.)
```
매번 확인해야 하는 필수 요소는 아니다. 하지만 아래 상황에서는 반드시 확인해야 한다.

- 인덱스를 기준으로 병합, 정렬, 필터링을 할 때
- 컬럼명이 예상과 다를 때
- 컬럼 선택 오류가 발생했을 때

즉, 문제가 발생했을 때 원인을 찾기 위한 확인 요소에 가깝다.

### 코드설명
1. columns: 컬럼 이름만 빠르게 확인할 때 사용한다.
2. index: 행의 기준이 되는 인덱스를 확인한다. (혹은 의미있는 키)

In [182]:
print("\n데이터 프레임의 인덱스 확인")
print(df.index)

print("\n데이터 프레임의 컬럼 목록 확인")
print(df.columns)


데이터 프레임의 인덱스 확인
RangeIndex(start=0, stop=4, step=1)

데이터 프레임의 컬럼 목록 확인
Index(['date', 'menu', 'price', 'qty', 'paid'], dtype='object')


## 2.3) 지금 이 데이터, 분석 가능한 상태인가? 판단 기준
EDA 초반에 던져야 할 질문은 이 데이터로 뭘 분석할까?가 아닌 지금 **이 상태로 분석이 가능한가?** 다.<br>
아래 조건을 대부분 만족하면 분석을 시작할 수 있는 상태라고 본다.
- 각 컬럼의 dtype이 의미에 맞다
- 결측치가 어디에 있는지 파악되어 있다
- 표 형태(DataFrame)가 유지되고 있다

이 조건이 만족되지 않으면 본 분석 전에 전처리가 필요하다.

# 3. dtype(자료형): 전처리의 핵심 엔진

## 3.1 dtype가 중요한 이유
dtype는 단순한 "자료형 정보"가 아니다. <br>
dtype란 Pandas가 해당 컬럼을 어떻게 계산하고, 비교하고, 해석할지를 결정하는 기준이자 규칙이다.

### 민약 dtype가 틀리다면?
1. 분명 보이는건 숫자인데 계산이 안 된다
2. 날짜인데 날짜 연산이 안 된다
3. True/False인데 필터가 이상하다

이 경우 문제는 값이 아니라 **Pandas가 그 값을 어떻게 해석하고 있는가**에 있다.

### 만약 dtype가 맞다면?
전처리에서 중요한 건 "원본 dtype"이 아닌 "분석 가능한 dtype" 이다.
- 가격 → 숫자형이어야 한다
- 날짜 → datetime이어야 한다
- 상태값 → bool 또는 범주형이어야 한다

dtype가 맞아야 그 다음 단계인 집계, 정렬, 필터링, 시각화가 의도한 대로 이어진다.<br>
때문에 항상 질문하는 습관을 가지자

## 3.2 dtype 변환의 두 가지 방식
dtype 변환은 아무 방식으로나 하면 되는 작업이 아니다.<br>
Pandas에서 dtype 변환은 다음 두 가지 방식으로만 정리된다.

In [183]:
# 예시 코드
import pandas as pd

df = pd.DataFrame({
    "date": ["2026-01-01", "2026/01/02", "2026-01-03"],
    "menu": ["Americano", "Latte", "Mocha"],
    "price": ["4500원", "5,000", None],
    "qty": ["2", 1, None],
    "paid": ["TRUE", "FALSE", True]
})

print("원본 테이블")
print(df)

print("\n 테이블 정보")
print(df.info())

원본 테이블
         date       menu  price   qty   paid
0  2026-01-01  Americano  4500원     2   TRUE
1  2026/01/02      Latte  5,000     1  FALSE
2  2026-01-03      Mocha   None  None   True

 테이블 정보
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    3 non-null      object
 1   menu    3 non-null      object
 2   price   2 non-null      object
 3   qty     2 non-null      object
 4   paid    3 non-null      object
dtypes: object(5)
memory usage: 248.0+ bytes
None


### 1. astype() — 강제 변환
타입을 바꿔서 다루겠다 는 선언<br>
이미 값이 "변환 가능한 형태"라고 확신할 수 있을 때 사용하는 방식이다.
- 문자열이지만 실제로는 전부 숫자일 때
- True / False 값이 일관되게 들어 있을 때

즉, 값의 형태가 깨끗하고 일관된다는 전제가 필요로 하다.

기본형태
``` python
컬럼.astype("string")
```
이 컬럼을 문자열 타입 기준으로 다루겠다고 Pandas에 알려주는 것이다.
- int, float, bool 도 가능

In [184]:
print("qty 컬럼을 astype(int)로 강제 변환 시도")
#df["qty"].astype(int)
print("실패하는 이유: 결측값이 존재하기 때문")

print("\nprice 컬럼을 astype(int)로 강제 변환 시도")
#df["price"].astype(int)
print("실패하는 이유: ""4500원"", ""5,000"" → 숫자로 해석 불가")

print("\nprice 컬럼을 astype(string)로 강제 변환 시도")
df["paid"] = (
    df["paid"]
    .astype("string")
    .str.upper() # Series에 문자열 전용 메소드를 쓰겠다는 접근자
    .map({"TRUE": True, "FALSE": False}) # 의미 → 값 매핑 ??????????????????????????????????
)
df["paid"]



qty 컬럼을 astype(int)로 강제 변환 시도
실패하는 이유: 결측값이 존재하기 때문

price 컬럼을 astype(int)로 강제 변환 시도
실패하는 이유: 4500원, 5,000 → 숫자로 해석 불가

price 컬럼을 astype(string)로 강제 변환 시도


0     True
1    False
2     True
Name: paid, dtype: bool

### 2. 파싱 변환 — 해석해서 바꾸는 방식
to_numeric, to_datetime 같은 함수는 값을 하나하나 해석해서 "숫자로 볼 수 있는지", "날짜로 볼 수 있는지"를 판단한다.<br>
전처리에서는 astype보다 이 방식이 더 자주 사용된다.
- 숫자처럼 보이는 문자열
- 날짜 형식이 섞여 있는 문자열

처럼 현실 데이터에 더 가까운 상황에서 사용한다.

#### to_datetime 문장 예시
``` python
# to_datetime
pd.to_datetime(컬럼, errors="coerce")
```
문자열 → 날짜/시간 변환 <br>
날짜처럼 보이는 값으로 월별, 요일별, 기간 분석을 하고 싶을 때 사용된다.

In [185]:
print("date 컬럼을 datetime으로 파싱 변환")
df["date"] = pd.to_datetime(df["date"], errors="coerce")
print(df["date"])

date 컬럼을 datetime으로 파싱 변환
0   2026-01-01
1          NaT
2   2026-01-03
Name: date, dtype: datetime64[ns]


#### to_numeric 문장 예시
```python
# to_numeric
pd.to_numeric(컬럼, errors="coerce")
```
문자열 → 숫자 변환 (전처리에서 가장 많이 씀) <br>
이 컬럼은 숫자처럼 보이지만 실제로는 문자열일 때 사용된다.

In [186]:
print("qty 컬럼을 to_numeric으로 파싱 변환")
df["qty"] = pd.to_numeric(df["qty"], errors="coerce")
print(df["qty"])

qty 컬럼을 to_numeric으로 파싱 변환
0    2.0
1    1.0
2    NaN
Name: qty, dtype: float64


In [187]:
print("price 컬럼 문자열 정리")
df["price"] = (
    df["price"]
    .astype("string")
    .str.replace(",", "", regex=False)
    .str.replace("원", "", regex=False)
)
df["price"] = pd.to_numeric(df["price"], errors="coerce")
print(df["price"])



price 컬럼 문자열 정리
0    4500
1    5000
2    <NA>
Name: price, dtype: Int64


## 3.3) 변환 실패를 처리하는 세 가지 errors 옵션
dtype 변환에서 가장 자주 마주치는 문제는 **변환 실패** 다.<br>
대부분의 데이터는 깔끔하지 않고 항상 맞지 않는데이터들이 존재한다.<br>
이러한 데이터를 errors 옵션을 통해 특정 값바꿔서 문제를 눈에 보이게 만든다.

이 옵션의 핵심 의미는 "문제를 숨기는 것"이 아닌 문제를 눈에 보이게 만드는 점이다.
- 어디서 변환이 실패했는지 확인할 수 있다
- 이후 결측치 처리 전략으로 이어질 수 있다

다음에 설명하는 3가지의 errors 옵션은 "변환이 실패하면 어떻게 할까?"를 묻고 실행한다. <br>
기억하자 값을 바꾸는것은 문제 해결이 아닌, 문제를 **눈에 보기에 만들기** 위함이다


### (1) errors="raise"
실패하면 즉시 중단
```python
pd.to_numeric(컬럼, errors="raise")
pd.to_datetime(컬럼, errors="raise")
```
변환에 실패하는 값이 하나라도 있으면 에러를 발생시키고 작업을 중단한다
- 데이터 검증 단계에는 유용
- 실무 전처리에는 거의 쓰이지 않음

### (2) errors="coerce"
실패하면 결측치로 바꾸고 계속 진행
```python
pd.to_numeric(컬럼, errors="coerce")
pd.to_datetime(컬럼, errors="coerce")
```
- 변환이 안 되는 값은 문제가 있는 데이터로 표시하고 일단 분석 흐름을 유지한다
    - 숫자 변환 실패 → NaN
    - 날짜 변환 실패 → NaT

문제가 되는 값은 바로 볼수 있기 때문에 다음 단계에서 의도적으로 처리한다.

In [188]:
print("qty의 결측값 확인")
print(df["qty"])

print("\ndate의 결측값 확인")
print(df["date"])

print("\nprice의 결측값 확인")
print(df["price"])

qty의 결측값 확인
0    2.0
1    1.0
2    NaN
Name: qty, dtype: float64

date의 결측값 확인
0   2026-01-01
1          NaT
2   2026-01-03
Name: date, dtype: datetime64[ns]

price의 결측값 확인
0    4500
1    5000
2    <NA>
Name: price, dtype: Int64


## 3.4) dtype 변환 후 확인의 중요성
dtype 변환은 코드를 실행했다고 끝나는 작업이 아니다. <br>
변환 이후에는 반드시 다시 dtypes 또는 info()를 해 의도대로 바뀌었는지 확인해야 한다.
- 의도한 dtype으로 바뀌었는지
- 의도하지 않은 결측값이 새로 생겼는지

를 확인하지 않으면 전처리는 "한 것처럼 보이기만 한 상태"가 된다.<br>
기억하자. 우리는 전처리를 하고 있는것이다.

In [189]:
print("바뀐 데이터프레임의 정보")
df.info()

바뀐 데이터프레임의 정보
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    2 non-null      datetime64[ns]
 1   menu    3 non-null      object        
 2   price   2 non-null      Int64         
 3   qty     2 non-null      float64       
 4   paid    3 non-null      bool          
dtypes: Int64(1), bool(1), datetime64[ns](1), float64(1), object(1)
memory usage: 230.0+ bytes


dtype 변환으로 qty, date, price, paid 4개의 데이터를 바꾸었으니 해당 컬럼의 타입을 확인하자.
- date → datetime64
- price, qty → float64 (결측치가 있으면 int가 아니라 float로 보이는 것이 정상)
- paid → bool

의도한대로 잘 바뀌었다고 볼수 있으나, 아직 처리해야할 문제<br>
**결측치**가 남아있다

# 4. 결측치(Missing Value) 처리의 논리

## 4.1) 결측치란 무엇인가?
결측치란 값이 비어 있거나, 알 수 없는 상태를 의미한다.
- 숫자형 결측 → NaN
- 날짜형 결측 → NaT

가 대표적인 예시이다

기억하자
- 결측치는 "이상한 데이터"가 아니라 현실 데이터를 그대로 반영한 상태다.


## 4.2) 결측치가 생기는 이유
1. 입력의 누락
    - 사람이 값을 입력하지 않음
2. 시스템 / 센서 오류
    - 수집 과정에서 값이 빠짐
3. 조인(merge) 실패
    - 다른 테이블과 매칭되지 않음
4. 원래 존재하지 않는 값
    - 옵션 미선택, 해당 항목 없음

결측치는 실수라기보다 데이터 생성 과정의 수많은 결과중 하나다.<br>
"왜 없을까?"를 생각해야 하는 대상이지 무조건 제거해야 할 대상은 아니다.

## 4.3) 결측치 처리의 기본 흐름
결측치 처리는 반드시 아래의 순서를 따라야 한다.
1. 결측치가 있는지 확인한다
2. 왜 생겼을지 추측한다
3. 분석 목적에 맞게 처리 전략을 고른다

이 순서를 건너뛰면 결과는 숫자로는 맞아 보여도 해석이 틀어질 가능성이 커진다.

In [190]:
df2=df.copy() # 기존 데이터프레임을 가져온다

print("바뀐 데이터프레임")
df2


바뀐 데이터프레임


,date,menu,price,qty,paid
0,2026-01-01,Americano,4500,2.0,True
1,NaT,Latte,5000,1.0,False
2,2026-01-03,Mocha,<NA>,NaN,True


## 4.4) 결측치의 확인
결측치 처리는 확인에서부터 시작한다.
- 어디에 결측치가 있는지
- 얼마나 있는지

이 단계에서는 아직 처리하지 않고, 현상만 정확히 파악하는 단계다.

대표적인 결측치 확인 함수
- isna(): 각 값이 결측치(NaN, NaT, None)인지 여부를 True / False로 반환하는 함수다.

In [191]:
print("date 컬럼 결측치 확인")
print(df["date"].isna())

print("\nprice 컬럼 결측치 확인")
print(df["price"].isna())

print("\nqty 컬럼 결측치 확인")
print(df["qty"].isna())


date 컬럼 결측치 확인
0    False
1     True
2    False
Name: date, dtype: bool

price 컬럼 결측치 확인
0    False
1    False
2     True
Name: price, dtype: bool

qty 컬럼 결측치 확인
0    False
1    False
2     True
Name: qty, dtype: bool


In [192]:
# tip notna()는 isna의 반대. 결측값이 아닌 값을 표현한다.
print("date 컬럼 결측치 확인(반전)")
print(df["date"].notna())

# tip isna().sum()을 통해 결측값의 갯수를 샐수도 있다.
print("\n컬럼별 결측치 확인")
print(df.isna().sum())

date 컬럼 결측치 확인(반전)
0     True
1    False
2     True
Name: date, dtype: bool

컬럼별 결측치 확인
date     1
menu     0
price    1
qty      1
paid     0
dtype: int64


In [193]:
# .any()를 사용해 결측치 존재여부만 빠르게 확인 가능하다.
print("date 컬럼에 결측치가 있는가?")
print(df["date"].isna().any())

print("\nprice 컬럼에 결측치가 있는가?")
print(df["price"].isna().any())

print("\nqty 컬럼 결측치 확인")
print(df["qty"].isna().any())

print("\nmenu 컬럼 결측치 확인")
print(df["menu"].isna().any())

date 컬럼에 결측치가 있는가?
True

price 컬럼에 결측치가 있는가?
True

qty 컬럼 결측치 확인
True

menu 컬럼 결측치 확인
False


## 4.5) 결측치 처리 전략 3가지
결측치 처리에는 정답이 없다.<br>
때문에 분석 목적에 따라 전략을 선택해야한다.

### 1. 삭제 (drop)
```python
df_drop = df.dropna()

```
결측치가 있는 행이나 컬럼을 아예 제거하는 방식이다.<br>
다음과 같은 상황에 사용하긴 하나 되도록 자제하도록 하자
- 결측 행이 매우 적을 때
- 결측이 분석 결과를 심각하게 왜곡할 때

특징
- 장점: 단순, 해석이 깔끔
- 단점: 데이터가 줄어들어 분석력이 떨어질 수 있음

가장 쉬운 방법이지만, 무조건적인 삭제는 정보 손실로 이어질 수 있기 때문에 신중하게 사용해야 한다.

In [194]:
print("결측치가 있는 행 삭제")
df_drop = df2.dropna()

print(df_drop)

결측치가 있는 행 삭제
        date       menu  price  qty  paid
0 2026-01-01  Americano   4500  2.0  True


### 2. 대체 (fill)
```python
df_fill = df.copy()

```
결측치를 의미 있는 값으로 채우는 방식이다.
- 데이터 양을 유지하고 싶을 때
- "비어 있음"이 어느 정도 의미를 가질 때

<br>

채워 넣는 값은 다양하지만, 대표적으로 다음과 같다.
- 평균, 중앙값, 최빈값
- 0, False, “Unknown”

대체값은 되도록이면 "없다" 라는 의미를 왜곡하지 않는 값이어야 한다.

In [195]:
df_fill = df.copy()

print("price 결측치를 0으로 대체")
df_fill["price"] = df_fill["price"].fillna(0)
print(df_fill)

print("\ndate 결측치를 특정 날짜로 대체")
df_fill["date"] = df_fill["date"].fillna(pd.Timestamp("2026-01-01"))
print(df_fill)

price 결측치를 0으로 대체
        date       menu  price  qty   paid
0 2026-01-01  Americano   4500  2.0   True
1        NaT      Latte   5000  1.0  False
2 2026-01-03      Mocha      0  NaN   True

date 결측치를 특정 날짜로 대체
        date       menu  price  qty   paid
0 2026-01-01  Americano   4500  2.0   True
1 2026-01-01      Latte   5000  1.0  False
2 2026-01-03      Mocha      0  NaN   True


In [196]:
print("컬럼별 결측치 확인")
print(df_fill.isna().sum())

컬럼별 결측치 확인
date     0
menu     0
price    0
qty      1
paid     0
dtype: int64


결측값을 fillna로 다른값으로 채운 price, date는 0인것을 확인할수 있다. 

다시한번 기억해야할것은 결측값은 신중하게 다뤄야 한다.<br>
자칫 잘못 다루면 오히려 결측값이 있을때 보다 못할수도 있다.

### 3. 추정 (개념만 이해)
다른 값들을 이용해 결측치를 추정하는 방식이다.
- 모델 기반 추정
- 주변 값 기반 추정

초심자 단계에서는 개념만 이해하고 넘어간다. <br>
실무에서도 항상 필요한 방법은 아니다.

# 5. 값 정제(Cleaning)의 대표 패턴
실무에서 전처리를 처음부터 매번 새로 생각하지 않고 자주 반복되는 패턴으로 처리할 수 있게 만드는 실력을 키워보자

In [197]:
import pandas as pd

df = pd.DataFrame({
    " Date ": ["2026-01-01", "2026-01-01", "2026-01-02", None,        "2026-01-02"],
    " Menu ": [" Latte ",   " latte",      "Americano ", "Mocha",     "Americano "],
    " Price ": ["5,000원",   "5,000원",     None,         "5,500원",   None],
    " Qty ":   [1,           1,            2,            None,        2],
    "   Paid": ["TRUE", "false", True, None, "1"],
    " memo ":  ["test",      "test",       "dup",        "x",         "dup"]
})

df


,Date,Menu,Price,Qty,Paid,memo
0,2026-01-01,Latte,"5,000원",1.0,TRUE,test
1,2026-01-01,latte,"5,000원",1.0,false,test
2,2026-01-02,Americano,None,2.0,True,dup
3,None,Mocha,"5,500원",NaN,None,x
4,2026-01-02,Americano,None,2.0,1,dup


## 5.1) 컬럼 구조 정리 (rename / drop)
전처리를 하기전 컬럼 이름을 안정적으로 만들고, 분석에 필요 없는 컬럼을 제거한다.

컬럼 정리의 목적
- 의미가 분명한 이름
- 불필요한 컬럼 제거
- 이후 코드 가독성 향상

In [198]:
print("=== 컬럼명 표준화 전 ===")
print(df.columns)

# 컬럼명 공백 제거 + 소문자 통일: " Date " -> "date"
df.columns = df.columns.str.strip().str.lower()

print("\n=== 컬럼명 표준화 후 ===")
print(df.columns)


=== 컬럼명 표준화 전 ===
Index([' Date ', ' Menu ', ' Price ', ' Qty ', '   Paid', ' memo '], dtype='object')

=== 컬럼명 표준화 후 ===
Index(['date', 'menu', 'price', 'qty', 'paid', 'memo'], dtype='object')


In [199]:
# 컬럼명 바꾸기: 의미가 더 명확한 이름으로
df = df.rename(columns={"menu": "menu_name"})
print(df.columns)

Index(['date', 'menu_name', 'price', 'qty', 'paid', 'memo'], dtype='object')


In [200]:
# 필요 없는 컬럼 제거: 분석에 안 쓰는 메모 컬럼 같은 것
df = df.drop(columns=["memo"])

print("=== 컬럼 정리 후 df ===")
print(df)
print("\n=== df.info() ===")
df.info()

=== 컬럼 정리 후 df ===
         date   menu_name   price  qty   paid
0  2026-01-01      Latte   5,000원  1.0   TRUE
1  2026-01-01       latte  5,000원  1.0  false
2  2026-01-02  Americano     None  2.0   True
3        None       Mocha  5,500원  NaN   None
4  2026-01-02  Americano     None  2.0      1

=== df.info() ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       4 non-null      object 
 1   menu_name  5 non-null      object 
 2   price      3 non-null      object 
 3   qty        4 non-null      float64
 4   paid       4 non-null      object 
dtypes: float64(1), object(4)
memory usage: 328.0+ bytes


In [201]:
# 최종 정리
df

,date,menu_name,price,qty,paid
0,2026-01-01,Latte,"5,000원",1.0,TRUE
1,2026-01-01,latte,"5,000원",1.0,false
2,2026-01-02,Americano,None,2.0,True
3,None,Mocha,"5,500원",NaN,None
4,2026-01-02,Americano,None,2.0,1


## 5.2) 문자열 정제 기본 4종 (공백 / 대소문자 / 불필요 문자 / 값 치환)
문자열 정제는 대부분 아래 4개의 문제들 중 하나다.
1. 공백 문제
2. 대소문자 문제
3. 불필요 문자
4. 값 치환 (다음에)

문자열 문제의 핵심은 “표기 흔들림”을 없애는 것이다.<br>
같은 값인데 다르게 저장되어 있으면 그룹핑/필터링/조인이 전부 깨진다.

### 공백 문제
### 대소문자 문제

In [202]:
print("=== menu_name 정제 ===")
df["menu_clean"] = (
    df["menu_name"]
    .astype("string")
    .str.strip()
    .str.lower()
)

df[["menu_name", "menu_clean"]]

=== menu_name 정제 ===


,menu_name,menu_clean
0,Latte,latte
1,latte,latte
2,Americano,americano
3,Mocha,mocha
4,Americano,americano


### 공백 문제
### 불필요 문자
- 공백 제거
- 쉼표 제거
- 단위 제거(원)

만 수행. 값 치환과 결측값은 다음에 수행한다.

In [203]:
print("\n=== price 문자열 정제 ===")
df["price_clean"] = (
    df["price"]
    .astype("string")
    .str.strip()
    .str.replace(",", "", regex=False)
    .str.replace("원", "", regex=False)
    
)

df[["price", "price_clean"]]


=== price 문자열 정제 ===


,price,price_clean
0,"5,000원",5000
1,"5,000원",5000
2,None,<NA>
3,"5,500원",5500
4,None,<NA>


## 5.3) 문자열 → 숫자 변환 표준 패턴 (to_numeric)
숫자 계산이 목적인데 값이 문자열이면 이 패턴을 적용한다.<br>
가격/금액처럼 “숫자여야 하는데 문자열로 들어온 컬럼”은 거의 항상 이 과정을 거친다.<br>

때문에 문자열 → 숫자 변환은 항상 같은 순서로 진행한다.
1. 문자열로 통일
2. 불필요한 문자 제거 (문자열 정제로 여기까지 수행)
3. 숫자로 파싱 (to_numeric)
4. 실패 값은 결측으로 표시
5. 결측 처리 전략 선택

In [204]:
print("=== price 숫자 파싱 ===")
df["price_clean"] = pd.to_numeric(df["price_clean"], errors="coerce")
print(df[["price", "price_clean"]])

print("\n=== price 결측치 개수 ===")
print(df["price_clean"].isna().sum())

=== price 숫자 파싱 ===
    price  price_clean
0  5,000원         5000
1  5,000원         5000
2    None         <NA>
3  5,500원         5500
4    None         <NA>

=== price 결측치 개수 ===
2


In [205]:
print("\n=== qty 숫자 파싱(안전 변환) ===")
df["qty_clean"] = pd.to_numeric(df["qty"], errors="coerce")

print(df[["qty", "qty_clean"]])
print("\n=== qty 결측치 개수 ===")
print(df["qty_clean"].isna().sum())



=== qty 숫자 파싱(안전 변환) ===
   qty  qty_clean
0  1.0        1.0
1  1.0        1.0
2  2.0        2.0
3  NaN        NaN
4  2.0        2.0

=== qty 결측치 개수 ===
1


### 결측 처리 전략 선택 (연습용: 0으로 대체)
연습에서는 0으로 채웠지만 실무에서는 "0이 의미 있는 값인가?"를 반드시 질문해야 한다.

In [206]:
print("\n=== 결측치 대체(연습용) ===")
df_fill = df.copy()
df_fill["price_clean"] = df_fill["price_clean"].fillna(0)
df_fill["qty_clean"]   = df_fill["qty_clean"].fillna(0)

df_fill[["price", "price_clean", "qty", "qty_clean"]]



=== 결측치 대체(연습용) ===


,price,price_clean,qty,qty_clean
0,"5,000원",5000,1.0,1.0
1,"5,000원",5000,1.0,1.0
2,None,0,2.0,2.0
3,"5,500원",5500,NaN,0.0
4,None,0,2.0,2.0


## 5.4) 범주값 통일 패턴 (TRUE / True / false 문제)
범주형 데이터는 값이 같아 보여도 표기가 다르면 다른 값으로 인식된다.
이 문제를 해결하지 않으면 그룹화, 집계 결과가 쪼개진다.

범주값 통일의 기본 원칙
- 문자열로 통일
- 공백/대소문자 차이를 제거
- 의미 기준으로 하나의 값으로 매핑
- 매핑 실패 값은 따로 확인

### 문자열로 통일
### 공백/대소문자 차이를 제거

In [207]:
print("=== paid 문자열 통일(정리 전) ===")
print(df_fill["paid"])

paid_norm = (
    df_fill["paid"]
    .astype("string")   # 문자열 기준으로 통일
    .str.strip()        # 앞뒤 공백 제거
    .str.upper()        # 대소문자 통일
)

print("\n=== paid_norm (정리 후) ===")
print(paid_norm)

=== paid 문자열 통일(정리 전) ===
0     TRUE
1    false
2     True
3     None
4        1
Name: paid, dtype: object

=== paid_norm (정리 후) ===
0     TRUE
1    FALSE
2     TRUE
3     <NA>
4        1
Name: paid, dtype: string


### 의미 기준으로 하나의 값으로 매핑

In [208]:
paid_map = {
    "TRUE": True,  "FALSE": False,
    "Y": True,     "N": False,
    "YES": True,   "NO": False,
    "1": True,     "0": False,
    "T": True,     "F": False,
}

df_fill["paid_bool"] = paid_norm.map(paid_map)

print("\n=== paid_bool 변환 결과 ===")
print(df_fill[["paid", "paid_bool"]])



=== paid_bool 변환 결과 ===
    paid paid_bool
0   TRUE      True
1  false     False
2   True      True
3   None       NaN
4      1      True


### 매핑 실패 값은 따로 확인

In [209]:
unknown = df_fill[df_fill["paid_bool"].isna()][["paid"]]
unknown

,paid
3,None


## 5.5) 중복 데이터의 판단과 처리 (duplicated / drop_duplicates)
중복 데이터는 값이 같은지보다 의미가 같은지가 중요하다. <br>
중복을 다룰 때는 옵션 2개만 기억하면 된다.
- subset: 무엇을 기준으로 중복인지 판단할지
- keep: first(첫 행 유지) / last(마지막 행 유지)

지금의 데이터를 예를 들어<br>
같은 날짜 + 같은 메뉴 + 같은 수량 + 같은 가격 + 같은 결제여부면 같은 주문이라고 가정한다.

In [210]:
df_fill

,date,menu_name,price,qty,paid,menu_clean,price_clean,qty_clean,paid_bool
0,2026-01-01,Latte,"5,000원",1.0,TRUE,latte,5000,1.0,True
1,2026-01-01,latte,"5,000원",1.0,false,latte,5000,1.0,False
2,2026-01-02,Americano,None,2.0,True,americano,0,2.0,True
3,None,Mocha,"5,500원",NaN,None,mocha,5500,0.0,NaN
4,2026-01-02,Americano,None,2.0,1,americano,0,2.0,True


In [211]:
df_dup = df_fill[["date", "menu_clean", "qty_clean", "price_clean", "paid_bool"]]
key_cols = ["date", "menu_clean", "qty_clean", "price_clean", "paid_bool"]

In [212]:
print("=== 중복 여부 마스크 ===")
dup_mask = df_dup.duplicated(subset=key_cols, keep="first")
print(dup_mask)


=== 중복 여부 마스크 ===
0    False
1    False
2    False
3    False
4     True
dtype: bool


In [213]:
print("\n=== 중복 제거 전 ===")
print(df_dup)

print("\n=== 중복 제거(첫 번째 유지) ===")
dedup_first = df_dup.drop_duplicates(subset=key_cols, keep="first")
print(dedup_first)

print("\n=== 중복 제거(마지막 유지) ===")
dedup_last = df_dup.drop_duplicates(subset=key_cols, keep="last")
print(dedup_last)


=== 중복 제거 전 ===
         date menu_clean  qty_clean  price_clean paid_bool
0  2026-01-01      latte        1.0         5000      True
1  2026-01-01      latte        1.0         5000     False
2  2026-01-02  americano        2.0            0      True
3        None      mocha        0.0         5500       NaN
4  2026-01-02  americano        2.0            0      True

=== 중복 제거(첫 번째 유지) ===
         date menu_clean  qty_clean  price_clean paid_bool
0  2026-01-01      latte        1.0         5000      True
1  2026-01-01      latte        1.0         5000     False
2  2026-01-02  americano        2.0            0      True
3        None      mocha        0.0         5500       NaN

=== 중복 제거(마지막 유지) ===
         date menu_clean  qty_clean  price_clean paid_bool
0  2026-01-01      latte        1.0         5000      True
1  2026-01-01      latte        1.0         5000     False
3        None      mocha        0.0         5500       NaN
4  2026-01-02  americano        2.0            0   

# 6. 인덱싱과 필터링: 필요한 데이터만 남기는 기술

## 6.1) 인덱싱이 중요한 이유
전처리는 "전체 데이터를 예쁘게 만드는 작업이 아닌 분석에 필요한 데이터만 정확히 남기는 작업이다. <br>
아무리 dtype, 결측치, 문자열 정제가 잘 되어 있어도 원하지 않는 행이나 컬럼이 섞여 있으면 분석 결과는 왜곡된다.

<br>
그래서 인덱싱은 전처리의 마지막 단계이자, 분석으로 넘어가기 직전 반드시 거쳐야 하는 관문이다.

## 6.2) Row 선택과 Column 선택의 구분
인덱싱을 헷갈리게 만드는 가장 큰 원인은 행과 열을 구분하지 않고 생각하는 것이다.

- 행(Row) 선택<br>
→ 어떤 관측치(사건, 기록) 를 볼 것인가

- 열(Column) 선택<br>
→ 어떤 변수(속성) 를 볼 것인가

이 둘은 목적이 완전히 다르다. 전처리 단계에서는
- "이 행이 분석 대상인가?"
- "이 컬럼이 분석에 필요한가?"

를 각각 독립적으로 판단해야 한다.

## 6.3) loc과 iloc의 차이 (정리 보강 + 슬라이싱 주의점 명확화)
인덱싱은 “데이터를 고르는 행위”이고, loc과 iloc은 그 선택 기준을 라벨로 할지, 위치로 할지 결정하는 방법이다.

<br>
loc과 iloc의 차이는 단순한 문법 차이가 아니라 접근 기준의 차이다.
- loc<br>
→ 라벨(이름) 기준 접근<br>
→ 컬럼명, 인덱스명으로 선택

- iloc<br>
→ 위치(순서) 기준 접근<br>
→ 몇 번째 행, 몇 번째 열로 선택

이 차이를 이해하지 못하면 "분명 맞는 것 같은데 결과가 다르다"는 상황이 자주 발생한다.

전처리/분석 코드에서는 의도가 명확한 loc 사용이 더 안전한 경우가 많다.

In [214]:
df = pd.DataFrame(
    {"menu": ["Latte", "Americano", "Mocha"],
    "price": [5000, 4500, 5500]},
    index=["A001", "A002", "A003"]
)
# 행의 이름(라벨): "A001", "A002", "A003"
# 행의 순서(위치): 0, 1, 2

### .loc : 라벨(이름) 기준 조회
언제 쓰는가?
- 인덱스나 컬럼 이름에 의미가 있을 때
- 조건 필터링과 함께 사용할 때

핵심 특징
- 행 / 열 이름으로 접근
- 문자열 라벨 사용 가능
- 조건 필터링과 궁합이 좋음
- 슬라이싱 시 끝 라벨이 포함될 수 있음

In [215]:
# 이름이 A002인 행을 가져와라
df.loc["A002"]

menu     Americano
price         4500
Name: A002, dtype: object

### .iloc : 위치(순서) 기준 조회
언제 쓰는가?
- “위에서 몇 번째”가 중요할 때
- 데이터 구조를 빠르게 확인할 때

핵심 특징
- 정수 위치만 사용
- 파이썬 리스트 슬라이싱과 동일
- 슬라이싱 시 끝은 항상 미포함

In [216]:
# 순서가 1인 행을 가져와라
df.iloc[1]

menu     Americano
price         4500
Name: A002, dtype: object

In [217]:
# 슬라이싱은 끝을 포함하지 않기 때문에 A001~A002만 나왔다.
df.iloc[0:2]

,menu,price
A001,Latte,5000
A002,Americano,4500


## 6.4) 인덱싱을 잘못 이해하면 생기는 문제
인덱싱을 잘못 쓰면 코드가 에러 없이 실행되는데도 결과만 조용히 틀리는 상황이 발생한다.
- 의도하지 않은 행이 선택된다
- 수정하려던 데이터가 원본에 반영되지 않는다
- 경고 없이 값이 부분적으로만 바뀐다

이런 문제는 "문법을 몰라서"가 아닌 지금 내가 무엇을 선택하고 있는지 인식하지 못해서 생긴다.<br>
그래서 인덱싱에서는 항상 아래 질문을 스스로에게 던져야 한다.
> “지금 내가 선택한게 행인가, 열인가, 아니면 둘 다인가?”

# 7. 조건 필터링(불리언 인덱싱)의 본질

## 7.1) 조건 필터링의 핵심 아이디어
조건 필터링의 핵심은 조건에 맞는 행을 고른다가 아니라, 조건식으로 True / False로 된 마스크를 만든다는 점이다.
1. 조건식 평가 → 각 행마다 True / False 생성
2. True인 행만 선택

예를 들어 "결제가 완료된 주문만 보고 싶다"면,
- 결제 완료 → True
- 결제 실패 → False

이렇게 만든 True/False 배열이 바로 필터(마스크) 다.

이 구조를 이해하지 못하면 왜 조건식이 먼저 계산되고, 왜 그 결과를 인덱싱에 쓰는지 이해하기 어렵다.

In [218]:
# 사용 데이터
import pandas as pd

df = pd.DataFrame({
    "order_id": [1, 2, 3, 4, 5, 6],
    "store": ["A","A","B","A","B","A"],
    "menu": ["Latte","Americano","Latte","Mocha","Latte","Latte"],
    "price": [5000, 4500, 5000, 5500, 5000, 5000],
    "paid": [True, False, True, True, False, True]
})

print("데이터프레임")
print(df)

print("\n데이터프레임.info()")
print(df.info())

데이터프레임
   order_id store       menu  price   paid
0         1     A      Latte   5000   True
1         2     A  Americano   4500  False
2         3     B      Latte   5000   True
3         4     A      Mocha   5500   True
4         5     B      Latte   5000  False
5         6     A      Latte   5000   True

데이터프레임.info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   order_id  6 non-null      int64 
 1   store     6 non-null      object
 2   menu      6 non-null      object
 3   price     6 non-null      int64 
 4   paid      6 non-null      bool  
dtypes: bool(1), int64(2), object(2)
memory usage: 326.0+ bytes
None


In [219]:
mask = (df["paid"] == True)

paid_orders = df[mask]

print("=== 필터 ===")
print(mask)

print("\n=== 필터 적용 전 ===")
print(df)
print(df.shape)

print("\n=== 필터 적용 후 ===")
print(paid_orders)
print(paid_orders.shape)

=== 필터 ===
0     True
1    False
2     True
3     True
4    False
5     True
Name: paid, dtype: bool

=== 필터 적용 전 ===
   order_id store       menu  price   paid
0         1     A      Latte   5000   True
1         2     A  Americano   4500  False
2         3     B      Latte   5000   True
3         4     A      Mocha   5500   True
4         5     B      Latte   5000  False
5         6     A      Latte   5000   True
(6, 5)

=== 필터 적용 후 ===
   order_id store   menu  price  paid
0         1     A  Latte   5000  True
2         3     B  Latte   5000  True
3         4     A  Mocha   5500  True
5         6     A  Latte   5000  True
(4, 5)


## 7.2) 조건을 따로 만드는 이유
초보자 입장에서는 이렇게 생각하기 쉽다. "바로 df[...]로 뽑으면 되지 않나?" 라고.<br>
하지만 실무에서는 3가지 이유 때문에 필터를 분리해두는 쪽이 훨씬 안전하다.

1. 조건에 맞는 행이 몇 개인지 바로 확인 가능
2. 여러 조건을 조합할 때 디버깅이 쉬움
3. 같은 조건을 재사용하기 쉬움

In [220]:
# 1. 조건에 맞는 행이 몇 개인지 바로 확인 가능
mask_paid = (df["paid"] == True)
print("조건에 맞는 행의 총 개수")
mask_paid.sum()   # True는 1로 계산됨 → 조건에 맞는 행 개수

조건에 맞는 행의 총 개수


np.int64(4)

In [221]:
# 2. 여러 조건을 조합할 때 디버깅이 쉬움
mask_storeA = (df["store"] == "A")
mask_latte  = (df["menu"] == "Latte")

mask_final = mask_paid & mask_storeA & mask_latte
mask_final


0     True
1    False
2    False
3    False
4    False
5     True
dtype: bool

In [222]:
# 3. 같은 조건을 재사용하기 쉬움
df[mask_final]
df.loc[mask_final, ["order_id", "menu", "price"]]


,order_id,menu,price
0,1,Latte,5000
5,6,Latte,5000


## 7.3) 다중 조건을 구성할때 필수 규칙
조건이 하나일 때는 괜찮지만, 두 개 이상이 되는 순간 실수가 급증한다. <br>
그래서 아래 규칙은 "이해"보다 습관처럼 쓰는 것이 중요하다.


### 규칙 1) &(AND), |(OR), ~(NOT) 사용 시 괄호 필수
&와 |는 연산자 우선순위가 낮기 때문에 괄호가 없으면 조건이 의도와 다르게 묶인다.<br>
때문에 괄호를 생략하면 파이썬이 연산 우선순위를 다르게 해석해서 에러 또는 엉뚱한 결과가 나온다.
- AND → (조건1) & (조건2)
- OR → (조건1) | (조건2)
- NOT → ~(조건)

<br>

### 규칙 2) and / or 가 아니라 & / | 를 사용한다
Pandas의 조건 필터링은 파이썬의 단일 True/False가 아니라 Series 단위의 True/False 연산이다.
- and, or <br>
→ 단일 True/False 판단용
- &, |<br>
→ 판다스의 True/False 배열(필터)용

In [223]:
# AND 조건
mask_and = (df["paid"] == True) & (df["store"] == "A") & (df["menu"] == "Latte")

# OR 조건
mask_or = (df["menu"] == "Latte") | (df["menu"] == "Mocha")

# NOT 조건
mask_not = ~(df["paid"] == True)

print("=== 필터 적용 전 ===")
print(df)

print("\n=== AND 조건 ===")
print(df.loc[mask_and])

print("\n=== OR 조건 ===")
print(df.loc[mask_or])

print("\n=== NOT 조건 ===")
print(df.loc[mask_not])

=== 필터 적용 전 ===
   order_id store       menu  price   paid
0         1     A      Latte   5000   True
1         2     A  Americano   4500  False
2         3     B      Latte   5000   True
3         4     A      Mocha   5500   True
4         5     B      Latte   5000  False
5         6     A      Latte   5000   True

=== AND 조건 ===
   order_id store   menu  price  paid
0         1     A  Latte   5000  True
5         6     A  Latte   5000  True

=== OR 조건 ===
   order_id store   menu  price   paid
0         1     A  Latte   5000   True
2         3     B  Latte   5000   True
3         4     A  Mocha   5500   True
4         5     B  Latte   5000  False
5         6     A  Latte   5000   True

=== NOT 조건 ===
   order_id store       menu  price   paid
1         2     A  Americano   4500  False
4         5     B      Latte   5000  False


## 7.4) 가장 안전한 기본 형태
조건 필터링에서 실무적으로 가장 안전하고 권장되는 형태는 다음 구조다.

```python
# 행 조건은 앞에, 컬럼 선택은 뒤에
df.loc[조건, 컬럼]
```

조건 필터링의 실무 표준 문장은 이 형태다.
- 조건 → 어떤 행을 고를지
- 컬럼 → 어떤 열만 볼지

In [224]:
# 1단계 조건 생성
condition = (df["paid"] == True)

# 2단계. 컬럼 정하기 (보여줄 열)
cols = ["menu", "price"]

# 3단계. 기본 형태로 적용: 이 한 줄로 "결제 완료인 행만 + 필요한 컬럼만" 표현 가능
result = df.loc[condition, cols]


print("=== 필터 적용 전 ===")
print(df)


print("\n=== 필터 ===")
print(condition)

print("\n=== 필터 적용 후 ===")
print(result)

=== 필터 적용 전 ===
   order_id store       menu  price   paid
0         1     A      Latte   5000   True
1         2     A  Americano   4500  False
2         3     B      Latte   5000   True
3         4     A      Mocha   5500   True
4         5     B      Latte   5000  False
5         6     A      Latte   5000   True

=== 필터 ===
0     True
1    False
2     True
3     True
4    False
5     True
Name: paid, dtype: bool

=== 필터 적용 후 ===
    menu  price
0  Latte   5000
2  Latte   5000
3  Mocha   5500
5  Latte   5000


# 8) 정렬과 표 형태 유지의 중요성

## 8.1) 보고서/리포트는 “정렬된 표”가 기본
실무에서 보고서는 보통 아래 요구를 포함한다.
- 최신순: 최근 데이터부터 보여주기 (날짜 기준 내림차순)
- 매출 큰 순: 매출 상위 메뉴/지점 찾기 (매출 기준 내림차순)
- 수량 많은 순: TOP N 만들기 (수량 기준 내림차순)

정렬이 하는 일은 단순히 보기 좋게 만드는 게 아닌, 우선순위를 표 위에 고정해서 의사결정이 가능하게 만드는 것이다.

In [225]:
import pandas as pd

df = pd.DataFrame({
    "date": ["2026-01-03", "2026-01-01", "2026-01-02", "2026-01-03", "2026-01-01"],
    "menu": ["Latte", "Americano", "Mocha", "Americano", "Latte"],
    "qty":  [2, 1, 3, 1, 1],
    "price":[5000, 4500, 5500, 4500, 5000]
})

df

,date,menu,qty,price
0,2026-01-03,Latte,2,5000
1,2026-01-01,Americano,1,4500
2,2026-01-02,Mocha,3,5500
3,2026-01-03,Americano,1,4500
4,2026-01-01,Latte,1,5000


- 날짜 순서가 뒤죽박죽
- 같은 메뉴가 흩어져 있음
- 패턴이 잘 안 보임

**정렬 없이 보면 (의미 파악 어려움)**

In [226]:
df.sort_values("date")

,date,menu,qty,price
1,2026-01-01,Americano,1,4500
4,2026-01-01,Latte,1,5000
2,2026-01-02,Mocha,3,5500
0,2026-01-03,Latte,2,5000
3,2026-01-03,Americano,1,4500


- 데이터의 시간 흐름이 보이기 시작
- 언제 어떤 메뉴가 나왔는지 파악 가능

## 8.2) sort_values vs sort_index 차이(감 잡기)
정렬은 기준이 두 가지로 갈린다.
- sort_values → "값 기준 정렬"
- sort_index  → "인덱스 기준 정렬"

### sort_values: 값 기준 정렬
매출, 수량, 가격처럼 “컬럼의 값”으로 순위를 만들 때 사용

```python
# 기본 구조
df.sort_values(by="컬럼명", ascending=True)

# 여러 럴럼
df.sort_values(by=["컬럼1", "컬럼2"], ascending=[False, True])
```
- 값(column 값) 기준으로 행 순서를 정렬한다
- 랭킹, 매출순, 수량순 같은 리포트용 정렬에 사용

In [227]:
# sort_values: 값 기준 정렬
print("=== 정렬 전 ===")
print(df)

print("\n=== 정렬 후 ===")
print(df.sort_values("price", ascending=False))

=== 정렬 전 ===
         date       menu  qty  price
0  2026-01-03      Latte    2   5000
1  2026-01-01  Americano    1   4500
2  2026-01-02      Mocha    3   5500
3  2026-01-03  Americano    1   4500
4  2026-01-01      Latte    1   5000

=== 정렬 후 ===
         date       menu  qty  price
2  2026-01-02      Mocha    3   5500
0  2026-01-03      Latte    2   5000
4  2026-01-01      Latte    1   5000
1  2026-01-01  Americano    1   4500
3  2026-01-03  Americano    1   4500


### sort_index: 인덱스 기준 정렬
날짜를 인덱스로 두었거나, groupby 결과처럼 인덱스가 의미를 가질 때 사용

```python
# 기본 구조
df.sort_index(ascending=True)
```
- 인덱스(행 라벨) 기준으로 순서를 정렬한다
- 날짜 인덱스, groupby 결과 정리에 자주 사용

In [228]:
# sort_index: 인덱스 기준 정렬
print("=== 정렬 전 ===")
print(df)

print("\n=== 정렬 후 ===")
df_date_indexed = df.set_index("date")
print(df_date_indexed.sort_index())


=== 정렬 전 ===
         date       menu  qty  price
0  2026-01-03      Latte    2   5000
1  2026-01-01  Americano    1   4500
2  2026-01-02      Mocha    3   5500
3  2026-01-03  Americano    1   4500
4  2026-01-01      Latte    1   5000

=== 정렬 후 ===
                 menu  qty  price
date                             
2026-01-01  Americano    1   4500
2026-01-01      Latte    1   5000
2026-01-02      Mocha    3   5500
2026-01-03      Latte    2   5000
2026-01-03  Americano    1   4500


### (추가) groupby 결과 인덱스 정리

In [229]:
orders = pd.DataFrame({
    "menu": ["Latte","Americano","Latte","Mocha","Mocha","Tea"],
    "qty":  [2, 1, 3, 1, 2, 4]
})
qty_sum = orders.groupby("menu")["qty"].sum()
qty_sum_sorted_by_index = qty_sum.sort_index()

print("=== 정렬 전 ===")
print(orders)

print("\n=== 정렬 후 ===")
print(qty_sum_sorted_by_index)


=== 정렬 전 ===
        menu  qty
0      Latte    2
1  Americano    1
2      Latte    3
3      Mocha    1
4      Mocha    2
5        Tea    4

=== 정렬 후 ===
menu
Americano    1
Latte        5
Mocha        3
Tea          4
Name: qty, dtype: int64


## 8.3) "표 형태 유지"가 중요한 이유
전처리와 분석에서는 표 형태(DataFrame)를 유지하는 것이 매우 중요하다.

표 형태를 유지하면 다음이 가능해진다.
- 컬럼 기반 연산이 자연스럽다
- GroupBy / Merge / Pivot이 안정적이다
- 시각화 단계로 바로 이어진다

반대로 Series로 바뀌면 생기는 문제가 있다.
- 컬럼 이름이 사라진다
- 다음 연산에서 형태가 예측되지 않는다
- 코드가 분기되기 시작한다

그래서 실무에서는 “Series로 변하는 순간”을 항상 의식해야 한다.

In [230]:
# ---------------------------------------
#- 결과는 Series, 컬럼 이름이 없음
#- 다음 단계에서 불편해짐
# ---------------------------------------

menu_qty = df.groupby("menu")["qty"].sum()
menu_qty

menu
Americano    2
Latte        3
Mocha        3
Name: qty, dtype: int64

In [231]:
# ---------------------------------------
# DataFrame 유지, 컬럼명 존재
# 다음 연산 / 정렬 / 시각화가 쉬움
# ---------------------------------------

menu_qty_df = df.groupby("menu")[["qty"]].sum()
menu_qty_df

,qty
menu,
Americano,2
Latte,3
Mocha,3


## 8.4) 초보자에게 추천하는 2가지 습관

정렬과 표 형태를 안정적으로 유지하기 위한
가장 효과적인 습관 두 가지다.

### 습관 1) 결과를 바로 정렬한다
- GroupBy 결과
- 집계 결과
- 요약 테이블

은 만들자마자 의도에 맞게 정렬한다. <br>
정렬을 미루면 중간 단계에서 해석이 흐려진다.

In [232]:
# ---------------------------------------
# 요약 → 정렬까지 한 번에
# 결과 해석이 즉시 가능
# ---------------------------------------

menu_summary = (
    df.groupby("menu")[["qty", "price"]]
        .sum()
        .sort_values("qty", ascending=False)
)

menu_summary

,qty,price
menu,,
Latte,3,10000
Mocha,3,5500
Americano,2,9000


### 습관 2) DataFrame 형태를 유지하려고 의식한다
컬럼 선택 시 이중 대괄호 사용 집계 결과가 Series인지 확인<br>
다음 단계에서 쓸 수 있는 형태인지 점검

In [233]:
# ---------------------------------------
# 단일 컬럼 선택 (Series)
# df.groupby("menu")["price"].mean()
# 단일 컬럼 선택은 데이터 클리닝이 아니라면 자제하도록 하자
# ---------------------------------------

df.groupby("menu")[["price"]].mean()


,price
menu,
Americano,4500.0
Latte,5000.0
Mocha,5500.0


# 9. 파생 피처 생성 (전처리의 목적 지점)

## 9.1 파생 피처란 무엇인가
파생 피처란 기존 컬럼을 가공해, 분석에 유용한 새로운 컬럼을 만드는 것이다.<br>
중요한 점은 파생 피처는 "새로운 정보를 창조"하는 게 아닌<br>
이미 있는 정보를 분석에 쓰기 좋은 형태로 바꾸는 작업이라는 것이다.

## 9.2 파생 피처는 언제 필요한가?
원본 데이터는 대개 기록 목적으로 저장되어 있다.<br>
하지만 분석은 비교·그룹·집계를 목적으로 한다.

그래서 다음과 같은 상황에서 파생 피처가 필요해진다.
- 원본 값 그대로는 비교 기준이 애매할 때
- 그룹 기준을 만들기 어려울 때
- 집계 질문을 바로 던질 수 없을 때

즉, "이 컬럼으로 바로 질문을 던질 수 있는가?"에 망설임이 생기면 파생 피처를 만들 시점이다.


### 기억해야할 파생 피처 포인트
파생 피처에 사용할 피처들은 기본적으로 분석 가능한 dtype 상태여야 한다.<br>
즉, 값의 정제가 어느정도 진행된 뒤에 해야하고 항상 파생 피처의 생성은 후순위 이다.


In [234]:
import pandas as pd

df = pd.DataFrame({
    "order_id": [1, 2, 3, 4, 5],
    "order_date": [ "2026-01-01", "2026-01-01", "2026-01-02", "2026-02-01", "2026-02-03"],
    "menu": ["Latte", "Americano", "Latte", "Mocha", "Latte"],
    "price": [5000, 4500, 5000, 5500, 5000],
    "qty": [1, 2, 1, 1, 3]
})

df["order_date"] = pd.to_datetime(df["order_date"]) #미리 데이트타임으로 형변환
print(df)
print("\n")
print(df.info())

   order_id order_date       menu  price  qty
0         1 2026-01-01      Latte   5000    1
1         2 2026-01-01  Americano   4500    2
2         3 2026-01-02      Latte   5000    1
3         4 2026-02-01      Mocha   5500    1
4         5 2026-02-03      Latte   5000    3


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   order_id    5 non-null      int64         
 1   order_date  5 non-null      datetime64[ns]
 2   menu        5 non-null      object        
 3   price       5 non-null      int64         
 4   qty         5 non-null      int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 328.0+ bytes
None


### 총 금액 파생 피처
현 데이터프레임으론 주문당 매출 이라는 질문에 대답할 수 없다.<br>
때문에 price와 qty 컬럼을 사용해 파생피처 sales를 만들고자 한다.

In [235]:
df["sales"] = df["price"] * df["qty"]

print(df)
print("\n")
print(df.info())

   order_id order_date       menu  price  qty  sales
0         1 2026-01-01      Latte   5000    1   5000
1         2 2026-01-01  Americano   4500    2   9000
2         3 2026-01-02      Latte   5000    1   5000
3         4 2026-02-01      Mocha   5500    1   5500
4         5 2026-02-03      Latte   5000    3  15000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   order_id    5 non-null      int64         
 1   order_date  5 non-null      datetime64[ns]
 2   menu        5 non-null      object        
 3   price       5 non-null      int64         
 4   qty         5 non-null      int64         
 5   sales       5 non-null      int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 368.0+ bytes
None


## 9.3) 날짜 파생 피처의 대표 예
날짜 데이터는 그 자체로도 의미가 있지만, 파생 피처를 만들었을 때 진짜 힘을 발휘한다. <br>
대표적인 날짜 파생 피처는 다음과 같다.
- 연(year)
- 월(month)
- 요일(day of week)

이 파생 피처들은
- 기간별 비교
- 패턴 탐색
- 트렌드 분석

의 기본 재료가 된다.

날짜 컬럼이 있다면 "그냥 두는 것"이 아니라 분해해서 쓰는 것이 기본이다.


### 주의사항
문자열 날짜 상태에서는 시간 분석의 절반 이상을 할 수 없다.<br>
때문에 to_datetime로 미리 문자열을 datetime으로 바꾸자

### 사용가능한 datetime 메소드
문장 구조
```python
datetime_타입_Series.dt.속성 # datetime 접근자 (문자열 접근자 .str 생각하기)

#또는

datetime_타입_Series.dt.메소드()
``` 
- 주의사항
    - 반드시 Series여야 한다
- 속성엔 다음과 같은 값이 들어올수 있다.
    - year
    - month
    - day
    - hour 등...
- 메소드엔 다양한 키워드가 들어갈수 있다.

In [236]:
df["year"] = df["order_date"].dt.year
df["month"] = df["order_date"].dt.month
df["weekday"] = df["order_date"].dt.day_name()

df[["order_date", "year", "month", "weekday"]]

,order_date,year,month,weekday
0,2026-01-01,2026,1,Thursday
1,2026-01-01,2026,1,Thursday
2,2026-01-02,2026,1,Friday
3,2026-02-01,2026,2,Sunday
4,2026-02-03,2026,2,Tuesday


## 9.4) 파생 피처의 목적: 파생 피처가 분석을 어떻게 바꾸는가
파생 피처의 목적은 명확하다.
> 분석 질문에 바로 답할 수 있는 형태로 데이터를 바꾸는 것

파생 피처가 잘 만들어지면 분석 코드는 단순해지고, 질문은 더 구체적으로 변한다.
- "날짜별 매출" → "월별 매출"
- "개별 주문" → "메뉴별 평균 가격"
- "원본 로그" → "비교 가능한 그룹"

반대로 파생 피처가 부족하면 분석 단계에서 복잡한 조건과 계산이 계속 반복된다.

## 9.5) 파생 피처 생성 시 주의할 점
파생 피처는 많이 만든다고 좋은 게 아니다. 다음 기준을 항상 함께 생각해야 한다.
- 이 컬럼이 분석 질문과 직접 연결되는가?
- 기존 컬럼으로도 충분하지는 않은가?
- 설명 가능한 파생 피처인가?

무분별한 파생 피처는 원본 데이터를 더럽힐 뿐만 아닌<br>
설명할 수 없는 파생 피처는 분석 결과를 해석하기 어렵게 만든다.

그래서 파생 피처는 "생각나는 대로 추가"가 아닌, 질문을 기준으로 최소한만 만드는 것이 중요하다.

# 10. 데이터 결합 (merge / join / concat)

## 10.1 데이터 결합이 필요한 이유
데이터 결합은 분석을 위한 선택이 아니라 분석이 가능해지기 위한 전제 조건이다.

실무 데이터는 하나의 테이블에 모든 정보가 들어 있는 경우가 거의 없다. <br>
보통 데이터는 다음처럼 나뉘어 있다.
- 거래/로그 테이블 → 언제, 무엇을 했는가
- 기준/마스터 테이블 → 그 값이 무엇을 의미하는가

그래서 분석을 하려면 여러 테이블을 결합해서 하나의 분석 단위로 만드는 과정이 필요하다.

## 10.2) merge = "키로 붙이는 조인"
```python
왼쪽_데이터프레임.merge(
    오른쪽_데이터프레임,
    on="공통_키",
    how="결합_방식")

```

merge는 공통 키(key)를 기준으로 테이블을 옆으로 붙이는 작업이다.<br>
그래서 키 선택이 틀리면 에러 없이 실행돼도 결과 전체가 틀어질 수 있다.


핵심 포인트
- 어떤 컬럼을 기준으로 붙일 것인가
- 이 키가 두 테이블에서 같은 의미를 가지는가

In [237]:
import pandas as pd

orders = pd.DataFrame([
    {"order_id": 1, "date": "2026-01-01", "menu": "Latte",     "sales": 5000, "customer_id": "C01"},
    {"order_id": 2, "date": "2026-01-01", "menu": "Americano", "sales": 4500, "customer_id": "C02"},
    {"order_id": 3, "date": "2026-01-02", "menu": "Cake",      "sales": 6000, "customer_id": "C03"},
])

menu_map = pd.DataFrame([
    {"menu": "Latte",     "category": "Coffee"},
    {"menu": "Americano", "category": "Coffee"},
    # Cake는 매핑 누락 상태
])

customers = pd.DataFrame([
    {"customer_id": "C01", "city": "Suwon",  "grade": "VIP"},
    {"customer_id": "C02", "city": "Yongin", "grade": "NEW"},
    {"customer_id": "C03", "city": "Suwon",  "grade": "NORMAL"},
])
print("주문 로그 데이터")
print(orders)

print("\n 메뉴 정보 데이터")
print(menu_map)

print("\n 고객 정보 데이터")
print(customers)


주문 로그 데이터
   order_id        date       menu  sales customer_id
0         1  2026-01-01      Latte   5000         C01
1         2  2026-01-01  Americano   4500         C02
2         3  2026-01-02       Cake   6000         C03

 메뉴 정보 데이터
        menu category
0      Latte   Coffee
1  Americano   Coffee

 고객 정보 데이터
  customer_id    city   grade
0         C01   Suwon     VIP
1         C02  Yongin     NEW
2         C03   Suwon  NORMAL


In [238]:
merged = orders.merge(
    menu_map,
    on="menu",
    how="left"
)

print(merged)


   order_id        date       menu  sales customer_id category
0         1  2026-01-01      Latte   5000         C01   Coffee
1         2  2026-01-01  Americano   4500         C02   Coffee
2         3  2026-01-02       Cake   6000         C03      NaN


orders를 기준으로
- menu를 키로
- menu_map의 category를 붙인다

merge의 결과로
- orders에는 없던 category 컬럼이 생긴다
- Cake처럼 매핑이 없는 값은 결측값이 된다

## 10.3) how 옵션의 의미
how는 결합 후에 어떤 행(row)을 남길지 결정하는 옵션이다. <br>
how = “기준을 어디에 두고 행을 남길 것인가”

- left<br>
→ 왼쪽 테이블의 행은 전부 유지

- inner<br>
→ 양쪽 테이블에 모두 존재하는 키만 유지

- outer<br>
→ 양쪽 테이블의 모든 행을 유지

### left — 왼쪽(원본) 유지 [LEFT JOIN]

```python
왼쪽.merge(오른쪽, on=키, how="left")

```
- 왼쪽 데이터프레임의 행은 전부 유지
- 오른쪽에 매핑이 없으면 NaN

언제 사용하나?
- 실무에서 가장 많이 쓰는 옵션
- 원본 로그는 보존하고
- 매핑 누락을 NaN으로 확인할 때

In [239]:
left_join = orders.merge(menu_map, on="menu", how="left")

print("주문 로그 데이터")
print(orders)

print("\n 메뉴 정보 데이터")
print(menu_map)

print("\n 주문 로그+ 메뉴 정보 데이터 (LEFT)")
print(left_join)

주문 로그 데이터
   order_id        date       menu  sales customer_id
0         1  2026-01-01      Latte   5000         C01
1         2  2026-01-01  Americano   4500         C02
2         3  2026-01-02       Cake   6000         C03

 메뉴 정보 데이터
        menu category
0      Latte   Coffee
1  Americano   Coffee

 주문 로그+ 메뉴 정보 데이터 (LEFT)
   order_id        date       menu  sales customer_id category
0         1  2026-01-01      Latte   5000         C01   Coffee
1         2  2026-01-01  Americano   4500         C02   Coffee
2         3  2026-01-02       Cake   6000         C03      NaN


### inner — 양쪽에 모두 있는 것만 [INNER JOIN]
```python
왼쪽.merge(오른쪽, on=키, how="inner")

```
- 양쪽 테이블에 공통으로 존재하는 키만 남김
- 매핑 안 되는 행은 아예 제거됨

언제 사용하나?
- 매핑이 제대로 됐는지
- 정합성 확인할 때

In [240]:
inner_join = orders.merge(menu_map, on="menu", how="inner")

print("주문 로그 데이터")
print(orders)

print("\n 메뉴 정보 데이터")
print(menu_map)

print("\n 주문 로그+ 메뉴 정보 데이터 (INNER)")
print(inner_join)


주문 로그 데이터
   order_id        date       menu  sales customer_id
0         1  2026-01-01      Latte   5000         C01
1         2  2026-01-01  Americano   4500         C02
2         3  2026-01-02       Cake   6000         C03

 메뉴 정보 데이터
        menu category
0      Latte   Coffee
1  Americano   Coffee

 주문 로그+ 메뉴 정보 데이터 (INNER)
   order_id        date       menu  sales customer_id category
0         1  2026-01-01      Latte   5000         C01   Coffee
1         2  2026-01-01  Americano   4500         C02   Coffee


### outer — 전체 합치기
```python
왼쪽.merge(오른쪽, on=키, how="outer")

```
- 양쪽 테이블의 모든 행을 전부 유지
- 한쪽에만 있으면 NaN

언제 사용하나?
- 어느 쪽에만 있는 값이 있는지
- 누락 / 신규 항목 탐색할 때

In [241]:
outer_join = orders.merge(menu_map, on="menu", how="outer")

print("주문 로그 데이터")
print(orders)

print("\n 메뉴 정보 데이터")
print(menu_map)

print("\n 주문 로그+ 메뉴 정보 데이터 (outer)")
print(outer_join)


주문 로그 데이터
   order_id        date       menu  sales customer_id
0         1  2026-01-01      Latte   5000         C01
1         2  2026-01-01  Americano   4500         C02
2         3  2026-01-02       Cake   6000         C03

 메뉴 정보 데이터
        menu category
0      Latte   Coffee
1  Americano   Coffee

 주문 로그+ 메뉴 정보 데이터 (outer)
   order_id        date       menu  sales customer_id category
0         2  2026-01-01  Americano   4500         C02   Coffee
1         3  2026-01-02       Cake   6000         C03      NaN
2         1  2026-01-01      Latte   5000         C01   Coffee


## 10.4) 결합 후 반드시 해야하는것
데이터 결합은 항상 검증이 필요한 작업이다. <br>
결합이 끝났다고 해서 "잘 붙었다"고 가정하면 위험하다.

반드시 확인해야 할 것은 이거다.
- 매핑이 안 된 행이 있는가?
- 예상보다 행 수가 줄거나 늘지 않았는가?

이를 확인하기 위한 가장 좋은 도구가 indicator 옵션이다.

indicator를 사용하면 각 행이 어떤 방식으로 결합되었는지 한눈에 알 수 있다.
- 왼쪽에만 있던 행
- 오른쪽에만 있던 행
- 정상적으로 매칭된 행

결합 결과에서 "어디서 문제가 생겼는지"를 바로 확인할 수 있다.

※ 이 아래에
merge(..., indicator=True) 예시를 넣기 좋다.

In [242]:
check = orders.merge(
    menu_map,
    on="menu",
    how="left",
    indicator=True
)

missing = check[check["_merge"] == "left_only"]
print(missing[["menu", "order_id"]])

   menu  order_id
2  Cake         3


indicator의 의미
- both → 양쪽에 있음
- left_only → 왼쪽에만 있음 (매핑 누락)
- right_only → 오른쪽에만 있음

# 11. 그룹화와 요약 테이블 만들기

## 11.1) 원본 로그와 요약 테이블의 차이
원본 로그와 요약 테이블은 역할 자체가 다르다.
- 원본 로그<br>
→ 사건(Event) 단위 데이터<br>
→ “무슨 일이 언제, 몇 번 발생했는가”
- 요약 테이블<br>
→ 비교(Comparison) 단위 데이터<br>
→ “그래서 전체적으로 어떤 경향이 있는가”

분석은 원본 로그를 그대로 읽는 게 아니라, 요약 테이블을 통해 패턴을 비교하는 과정이다. <br>
그래서 GroupBy는 "옵션 기능"이 아니라 분석 단계로 넘어가기 위한 필수 과정이다.



## 11.2) GroupBy의 역할
```python
데이터프레임.groupby("기준컬럼")["집계대상컬럼"].집계함수().reset_index(name="컬럼 이름")

```

GroupBy의 역할은 한 문장으로 정리할 수 있다.
- 기준을 나누고, 각 그룹을 집계한다.

GroupBy는
- 데이터를 나누고(split)
- 각 묶음에 대해 계산을 적용(apply)
- 그 결과를 다시 합친다(combine)

이 구조를 가진다.

그래서 GroupBy는 "값을 계산하는 함수"가 아닌 데이터 구조를 바꾸는 연산에 가깝다.

In [243]:
import pandas as pd

df = pd.DataFrame([
    {"date": "2026-01-01", "store": "광교점", "menu": "Americano", "sales": 9000},
    {"date": "2026-01-01", "store": "광교점", "menu": "Latte",     "sales": 5000},
    {"date": "2026-01-02", "store": "광교점", "menu": "Latte",     "sales": 5000},
    {"date": "2026-01-03", "store": "수원점", "menu": "Americano", "sales": 4500},
    {"date": "2026-01-03", "store": "수원점", "menu": "Mocha",     "sales": 5500},
])

df["date"] = pd.to_datetime(df["date"])
df["ym"] = df["date"].dt.to_period("M").astype(str) #????????????????????

store_report = df.groupby("store")["sales"].sum().reset_index(name="total_sales")
menu_report = df.groupby("menu")["sales"].mean().reset_index(name="avg")


print("원본 로그(주문 1건 단위):")
print(df)

print("\n매장 단위 요약(매장별 매출 합계):")
print(store_report)

print("\n메뉴 단위 요약(메뉴별 매출 합계):")
print(menu_report)

원본 로그(주문 1건 단위):
        date store       menu  sales       ym
0 2026-01-01   광교점  Americano   9000  2026-01
1 2026-01-01   광교점      Latte   5000  2026-01
2 2026-01-02   광교점      Latte   5000  2026-01
3 2026-01-03   수원점  Americano   4500  2026-01
4 2026-01-03   수원점      Mocha   5500  2026-01

매장 단위 요약(매장별 매출 합계):
  store  total_sales
0   광교점        19000
1   수원점        10000

메뉴 단위 요약(메뉴별 매출 합계):
        menu     avg
0  Americano  6750.0
1      Latte  5000.0
2      Mocha  5500.0


## 11.3) 집계 함수(aggregation) 핵심 5종
실무에서 사용하는 집계 함수는 생각보다 종류가 많지 않다.<br>
아래 다섯 가지만 제대로 쓰면 대부분의 요약 테이블을 만들 수 있다.

- sum: 총합 (총매출, 총수량)
- mean: 평균 (평균 매출, 평균 수량, 성공률)
- count: 개수 (주문 건수 = 행 개수)
- nunique: 고유값 개수 (고유 메뉴 수, 고유 고객 수)
- min / max: 최소/최대 (최소/최대 수량, 최대 매출 등)

중요한 건 "함수를 많이 아는 것"이 아니라 지금 어떤 질문을 하고 있는가다.

### agg란?
GroupBy 결과에 집계 함수 하나만 쓰는 경우는 드물다.<br>
대부분은
- 합계도 보고 싶고
- 평균도 보고 싶고
- 개수도 같이 보고 싶다.

이럴 때 필요한 게 agg다.

여러 집계를 한 번에, 명시적으로 관리할 수 있어 요약 테이블의 구조가 예측 가능해지고, 이후 후처리도 훨씬 쉬워진다.

In [244]:
import pandas as pd

df = pd.DataFrame([
    {"store": "광교점", "menu": "Americano", "qty": 2, "sales": 9000, "paid": True},
    {"store": "광교점", "menu": "Latte",     "qty": 1, "sales": 5000, "paid": True},
    {"store": "광교점", "menu": "Latte",     "qty": 2, "sales": 0,    "paid": False},
    {"store": "수원점", "menu": "Americano", "qty": 1, "sales": 4500, "paid": True},
    {"store": "수원점", "menu": "Mocha",     "qty": 1, "sales": 0,    "paid": False},
])

summary = df.groupby("store").agg(
    total_sales=("sales", "sum"),
    avg_sales=("sales", "mean"),
    orders=("menu", "count"),
    unique_menus=("menu", "nunique"),
    min_qty=("qty", "min"),
    max_qty=("qty", "max"),
    paid_rate=("paid", "mean"),
).reset_index()

print("사용한 집계 함수 출력")
print(summary)

사용한 집계 함수 출력
  store  total_sales    avg_sales  orders  unique_menus  min_qty  max_qty  \
0   광교점        14000  4666.666667       3             2        1        2   
1   수원점         4500  2250.000000       2             2        1        1   

   paid_rate  
0   0.666667  
1   0.500000  


## 11.4) MultiIndex가 뭐고 왜 불편한가?
GroupBy와 agg를 쓰다 보면 자연스럽게 MultiIndex가 생긴다.
MultiIndex 자체가 나쁜 건 아니다. 문제는 후처리 단계에서 불편하다는 점이다.
- 컬럼 접근이 복잡해진다
- 컬럼 이름이 길고 직관적이지 않다
- 이후 join, 시각화 단계에서 귀찮아진다

그래서 실무에서는 MultiIndex를 그대로 두기보다는 컬럼을 평평하게 만드는 후처리를 거의 항상 한다.

In [245]:
df.groupby("menu")["sales"].agg(["sum", "mean", "count"])
# 이렇게 되면 컬럼 접근이 불편해지고 rename / merge / 시각화에서 귀찮아진다.

,sum,mean,count
menu,,,
Americano,13500,6750.0,2
Latte,5000,2500.0,2
Mocha,0,0.0,1


In [246]:
summary = df.groupby("store").agg(
    total_sales=("sales", "sum"),
    avg_sales=("sales", "mean"),
    orders=("menu", "count"),
).reset_index()
summary
# 그래서 실무에서는 이런식으로 컬럼명을 직접 지정하는 agg 형태를 더 선호한다.

,store,total_sales,avg_sales,orders
0,광교점,14000,4666.666667,3
1,수원점,4500,2250.000000,2


# 12) 피벗(표 형태 변환) 이론: 리포트용 표 만들기
피벗은 계산을 새로 하는 작업이 아니라 이미 계산된 결과를 사람이 읽기 좋은 표 형태로 바꾸는 작업이다.
때문에 피벗은 전처리 후반부, 또는 GroupBy 이후에 등장한다.

## 12.1) 피벗이 필요한 순간
분석을 하다 보면 이런 요구가 반드시 생긴다.
- 기준 A × 기준 B 형태의 표
- 보고서에 바로 넣을 수 있는 표

이때 필요한 게 피벗이다.

피벗은 데이터를 계산하기 위해 쓰는 도구가 아닌, 결과를 정리하기 위해 쓰는 도구다.

In [247]:
import pandas as pd

# 예시 데이터(일부 조합이 반복되도록 일부러 2행 넣음: 2026-01 + Mon)
df = pd.DataFrame([
    {"ym": "2026-01", "day": "Mon", "sales": 100},
    {"ym": "2026-01", "day": "Mon", "sales": 50},   # 같은 조합이 2개라서 pivot이 바로 안 됨
    {"ym": "2026-01", "day": "Tue", "sales": 80},
    {"ym": "2026-02", "day": "Mon", "sales": 120},
    # 2026-02의 Tue가 없어서 피벗 후 NaN이 생길 수 있음
])

print("원본(long 형태)")
df


원본(long 형태)


,ym,day,sales
0,2026-01,Mon,100
1,2026-01,Mon,50
2,2026-01,Tue,80
3,2026-02,Mon,120


이 상태에서 바로 매장 비교도 어렵고, 메뉴별 비교도 한눈에 볼수 없다.

## 12.2) 긴 형태(long) → 넓은 형태(wide)
피벗을 이해하려면 데이터의 "형태"부터 이해해야 한다.

### 긴 형태 (long)
- 한 행 = 하나의 관측치
- 기준 값이 세로로 길게 늘어선 구조
- 분석과 계산에 유리함

대부분의 원본 데이터와 GroupBy 결과는 long 형태다.

### 넓은 형태 (wide)
- 기준 값이 열(column)로 펼쳐진 구조
- 비교와 보고서에 유리함
- 사람이 한눈에 보기 쉬움

피벗의 역할은 long 형태를 wide 형태로 바꾸는 것이다.

```python
데이터프레임.pivot(
    index="행 기준",
    columns="열 기준",
    values="값"
)

In [248]:
# 1단계: 먼저 요약 (칸당 값 1개 만들기)
summary = (
    df.groupby(["ym", "day"])["sales"]
    .sum()
    .reset_index()
)

print("\nGroupBy 요약 결과")
print(summary)

# 2단계: pivot으로 표 설계
report = summary.pivot(
    index="ym",
    columns="day",
    values="sales"
)

print("\npivot 결과")
print(report)

# 즉, pivot은 표 설계 도구다
# "이걸 행으로, 이걸 열로, 이 값을 채운다"라고 말하는 방식이다


GroupBy 요약 결과
        ym  day  sales
0  2026-01  Mon    150
1  2026-01  Tue     80
2  2026-02  Mon    120

pivot 결과
day        Mon   Tue
ym                  
2026-01  150.0  80.0
2026-02  120.0   NaN


## 12.3) pivot vs unstack은 뭐가 다른가?
pivot과 unstack은 결과는 비슷해 보이지만 출발점이 다르다.

pivot은 아직 GroupBy가 되지 않은 데이터에서 바로 표 형태를 만들 때 사용한다.
- index / columns / values를 명시적으로 지정
- 데이터 구조가 직관적으로 드러남
- 값이 하나로 유일해야 한다

<br>

unstack은 이미 GroupBy로 집계된 결과를 다시 펼치는 용도다.
- MultiIndex를 열로 풀어낸다
- GroupBy 이후 후처리 단계에서 등장
- index 구조를 이해해야 쓰기 쉽다

<br>

한 줄로 정리하면
- 원본 데이터에서 바로 표 만들기 → pivot
- GroupBy 결과를 펼치기 → unstack

In [249]:
# GroupBy 결과 (MultiIndex Series)
multi = df.groupby(["ym", "day"])["sales"].sum()

print("\nGroupBy 결과 (MultiIndex)")
print(multi)

# unstack으로 인덱스 레벨을 열로 펼침
wide = multi.unstack()

print("\nunstack 결과")
print(wide)
# 즉, unstack은 이미 만들어진 구조를 펼치는 도구다


GroupBy 결과 (MultiIndex)
ym       day
2026-01  Mon    150
         Tue     80
2026-02  Mon    120
Name: sales, dtype: int64

unstack 결과
day        Mon   Tue
ym                  
2026-01  150.0  80.0
2026-02  120.0   NaN


## 12.4) 피벗에서 자주 만나는 실수 포인트 2개
피벗은 문법보다 데이터 상태를 이해하지 못해서 실패하는 경우가 많다.

In [250]:
import pandas as pd

df = pd.DataFrame([
    {"date": "2026-01-01", "menu": "Americano", "store": "광교점", "sales": 9000},
    {"date": "2026-01-02", "menu": "Latte",     "store": "광교점", "sales": 5000},
    {"date": "2026-01-03", "menu": "Latte",     "store": "수원점", "sales": 5000},
    {"date": "2026-01-06", "menu": "Americano", "store": "광교점", "sales": 4500},
    {"date": "2026-02-03", "menu": "Mocha",     "store": "수원점", "sales": 5500},
])

df["date"] = pd.to_datetime(df["date"])
df

,date,menu,store,sales
0,2026-01-01,Americano,광교점,9000
1,2026-01-02,Latte,광교점,5000
2,2026-01-03,Latte,수원점,5000
3,2026-01-06,Americano,광교점,4500
4,2026-02-03,Mocha,수원점,5500


### 값이 여러 개라서 피벗이 안 되는 경우
pivot은 한 index × 한 column 조합에 값이 하나만 존재해야 한다. <br>
만약 값이 여러 개일 경우 피벗은 실패한다.

이 상황은 보통 "아직 집계가 안 된 데이터"에서 발생한다.<br>
해결법은 2가지가 있다.
1. GroupBy로 집계한다. (groupby + unstack)
2. pivot_table 을 사용한다.


In [251]:
menu_store_sales = (
    df.groupby(["menu", "store"])["sales"]
    .sum()
    .reset_index()
)

menu_store_sales.pivot(
    index="menu",
    columns="store",
    values="sales"
)

print("GroupBy를 한 피봇")
menu_store_sales

GroupBy를 한 피봇


,menu,store,sales
0,Americano,광교점,13500
1,Latte,광교점,5000
2,Latte,수원점,5000
3,Mocha,수원점,5500


In [252]:
report = df.pivot_table(
    index="menu",
    columns="store",
    values="sales",
    aggfunc="sum"
)

print("pivot_table을 사용한 피봇")
report

pivot_table을 사용한 피봇


store,광교점,수원점
menu,,
Americano,13500.0,NaN
Latte,5000.0,5000.0
Mocha,NaN,5500.0


### 빈 칸(NaN)이 생기는 것
피벗 결과에 NaN이 생긴다는 건 해당 조합의 데이터가 원래 없었다는 뜻이다. <br>
이건 에러가 아니라 정보지만 보고서나 시각화 단계에서는 처리가 필요할 수 있다.

0으로 채울것인지, 그대로 채울것인지 분석에 목적에 따라 달라진다.

In [253]:
df

,date,menu,store,sales
0,2026-01-01,Americano,광교점,9000
1,2026-01-02,Latte,광교점,5000
2,2026-01-03,Latte,수원점,5000
3,2026-01-06,Americano,광교점,4500
4,2026-02-03,Mocha,수원점,5500


In [254]:
pivot_result = df.pivot_table(
    index="menu",
    columns="store",
    values="sales",
    aggfunc="sum"
)

print("GroupBy를 한 피봇.fillna")
pivot_result.fillna(0)

GroupBy를 한 피봇.fillna


store,광교점,수원점
menu,,
Americano,13500.0,0.0
Latte,5000.0,5000.0
Mocha,0.0,5500.0


보강: 날짜 × 매장 피벗 (리포트용)

In [255]:
date_store_sales = (
    df.groupby(["date", "store"], )["sales"]
    .sum()
    .reset_index()
)

date_store_sales.pivot(    
    index="date",
    columns="store",
    values="sales").fillna(0)

store,광교점,수원점
date,,
2026-01-01,9000.0,0.0
2026-01-02,5000.0,0.0
2026-01-03,0.0,5000.0
2026-01-06,4500.0,0.0
2026-02-03,0.0,5500.0


In [256]:
df.pivot_table(
    index="date",
    columns="store",
    values="sales",
    aggfunc="sum"
).fillna(0)

store,광교점,수원점
date,,
2026-01-01,9000.0,0.0
2026-01-02,5000.0,0.0
2026-01-03,0.0,5000.0
2026-01-06,4500.0,0.0
2026-02-03,0.0,5500.0


# 13. apply vs map

## 13.1) map의 용도 — "값 치환 / 매핑"에 특화
``` python
Series.map(매핑_딕셔너리)
```
- 대상은 항상 단일 컬럼(Series)
- 기준은 딕셔너리의 key → value

<br>

map은 값 → 값으로 치환하는 단순 변환에 특화된 함수다.<br>
한 컬럼의 값들을 1:1로 바꿀 때 가장 적합하다.
- 범주값 매핑 (예: "Y" → True)
- 코드값 → 의미값 변환
- 이미 정의된 규칙에 따른 값 치환


In [257]:
import pandas as pd

df = pd.DataFrame({
    "channel": ["kiosk", "app", "kiosk", "web"]
})

channel_map = {
    "kiosk": "키오스크",
    "app": "앱",
    "web": "웹"
}
print("매핑 전")
print(df)

# channel 컬럼의 값들을 channel_map 기준으로 그대로 치환한다
df["channel_name"] = df["channel"].map(channel_map)
print("\n  매핑 후")
print(df)

매핑 전
  channel
0   kiosk
1     app
2   kiosk
3     web

  매핑 후
  channel channel_name
0   kiosk         키오스크
1     app            앱
2   kiosk         키오스크
3     web            웹


### 자주하는 실수포인트
- 매핑표에 없는 값은 NaN이 된다
- 그래서 항상 누락을 한 번 확인하는 게 좋다

In [258]:
missing = df[df["channel_name"].isna()]
print(missing)

Empty DataFrame
Columns: [channel, channel_name]
Index: []


## 13.2) apply의 용도
```python
Series.apply(함수)
# 또는
DataFrame.apply(함수, axis=1)
```
- 한 값씩 함수에 넣어서 처리
- 단일 컬럼 기준 로직

<br>

apply는 조건과 로직이 필요한 규칙 기반 처리에 사용한다.<br>
단순 치환으로는 표현할 수 없는 경우, 즉 아래 상황에서 apply가 필요해진다.
- 값의 범위에 따라 결과가 달라질 때
- 여러 조건을 동시에 판단해야 할 때
- 계산 과정이 여러 단계일 때



In [259]:
import pandas as pd

df = pd.DataFrame({"sales": [0, 3000, 12000]})

def grade_sales(x):
    if x == 0:
        return "C"
    elif x < 10000:
        return "B"
    else:
        return "A"

df["grade"] = df["sales"].apply(grade_sales)
print(df)

   sales grade
0      0     C
1   3000     B
2  12000     A


In [260]:
import pandas as pd

df = pd.DataFrame([
    {"sales": 12000, "paid": True},
    {"sales": 5000,  "paid": True},
    {"sales": 8000,  "paid": False},
])


def final_label(row):
    if row["paid"] is False:
        return "FAIL"
    if row["sales"] >= 10000:
        return "A"
    return "B"


print("행 단위")
df["label"] = df.apply(final_label, axis=1)
print(df)

행 단위
   sales   paid label
0  12000   True     A
1   5000   True     B
2   8000  False  FAIL


### apply 사용 시 주의점
apply는 강력하지만 주의해서 사용해야 한다.
- 처리 속도가 느리다
- 코드가 길어지고 복잡해지기 쉽다
- 벡터화된 Pandas 연산의 장점을 잃는다

그래서 apply는 가장 먼저 쓰는 도구가 아니라, 마지막에 선택하는 도구다.

## 13.4) 선택 기준 요약
apply와 map의 선택 기준은 아주 간단하게 정리할 수 있다.
- 값 → 값 치환이면 map
- 판단·조건·계산 로직이 필요하면 apply

한 줄로 정리하면 이렇다.<br>
매핑이면 map, 판단이 필요하면 apply


# 14. 파일 입출력과 작업 마무리

(출처: 1회차 6)

12.1 파일 입출력이 중요한 이유
12.2 CSV / Excel / JSON 사용 기준
12.3 전처리 결과물은 어떤 형태여야 하는가

## 14. 파일 입출력과 작업 마무리
13.1 파일 입출력이 중요한 이유

전처리는
항상 저장과 재사용을 전제로 한다.

13.2 파일 형식 선택 기준

CSV는 기본,
Excel은 공유,
JSON은 구조 데이터에 적합하다.

13.3 전처리 결과물의 기준

바로 집계·시각화가 가능한
“분석용 테이블” 상태여야 한다.

13. 파일 입출력과 작업 마무리

## 14.1) 파일 입출력이 중요한 이유
전처리는 일회성 작업이 아니라 재사용을 전제로 한 작업이다.<br>
분석에서 파일 입출력은 단순한 저장 기능이 아니다.
- 전처리 결과를 보존한다
- 분석 / 시각화 / 리포트 단계와 작업을 분리하기 위해
- 팀원 또는 미래의 나와 결과를 공유하기 위해

파일 저장은 분석의 부수 작업이 아닌 분석 과정의 일부다.

## 14.2) 파일 형식 선택 기준
파일 형식은 편한 것이 아닌 사용 목적에 따라 선택해야 한다.

In [261]:
import os
os.makedirs("data", exist_ok=True)

### CSV
분석 결과를 전달한다 → CSV가 기본 선택이다.
- 가장 단순한 표 형태
- 거의 모든 도구에서 읽을 수 있음
- 분석 결과 공유용으로 가장 많이 사용

In [262]:
csv_path = "data/sales.csv"

df.to_csv(csv_path, index=False, encoding="utf-8-sig")
df_csv = pd.read_csv(csv_path)

# index=False
# 인덱스가 의미 있는 데이터가 아니라면 파일에 저장하지 않는다.

# encoding="utf-8-sig"
# 엑셀에서 한글이 깨지는 문제를 줄이기 위해서 변경

### Excel (xlsx)
"사람이 직접 본다" → Excel이 편하다.
- 사람이 직접 열어보는 용도
- 보고용, 검토용 데이터에 적합
- 여러 시트를 사용할 수 있음

In [263]:
excel_path = "data/sales.xlsx"

df.to_excel(excel_path, index=False)
df_xlsx = pd.read_excel(excel_path)

### JSON
"시스템 간 데이터 교환"→ JSON을 쓴다.
- 구조화된 데이터 형식
- API, 서비스 연동, 로그 저장에 자주 사용
- 테이블보다는 데이터 구조 전달에 강함

In [264]:
json_path = "data/sales.json"

df.to_json(
    json_path,
    orient="records",
    force_ascii=False,
    indent=2
)
df_json = pd.read_json(json_path)


#### orient="records"?

이 옵션은 JSON을 다음 형태로 만든다.

```json
[
  {"date": "2026-01-01", "menu": "아메리카노", "sales": 9000},
  {"date": "2026-01-02", "menu": "라떼", "sales": 5000}
]
```

## 14.3) 저장 전에 반드시 점검할 체크리스트
- 이 테이블로 바로 GroupBy가 가능한가?
- 피벗이나 시각화를 바로 할 수 있는가?
- 컬럼 이름만 보고 의미가 전달되는가?

이 질문에 모두 "예"라고 답할 수 있을 때 저장한다.

전처리 마무리를 짓는다.